In [3]:
%%configure -f
{
    "conf": {
        "spark.jars": "s3://sjet-datamart-bucket/Jars/mssql-jdbc-6.1.0.jre8.jar"        
    }
}

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
4,application_1673498272080_0025,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
2,application_1673498272080_0023,pyspark,dead,Link,Link,
4,application_1673498272080_0025,pyspark,idle,Link,Link,✔


In [4]:
UNION_DF = spark.sql('''

                        SELECT * FROM precogdb.df5_red_high_rate_of_descent
                        where date > '2021-10-31'
                        UNION
                        SELECT * FROM precogdb.df5_amber_high_rate_of_descent
                        where date > '2021-10-31'
                        UNION
                        SELECT * FROM precogdb.df5_green_high_rate_of_descent
                        where flightdate > '2021-10-31'
                        
                        
                        
                        ''')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
spark.sql('''
SELECT * FROM precogdb.df5_green_high_rate_of_descent
''').printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- exceedanceid: long (nullable = true)
 |-- exceedance: string (nullable = true)
 |-- phaseid: long (nullable = true)
 |-- phase: string (nullable = true)
 |-- ecn: string (nullable = true)
 |-- employeename: string (nullable = true)
 |-- rank: string (nullable = true)
 |-- tailno: string (nullable = true)
 |-- departure: string (nullable = true)
 |-- arrival: string (nullable = true)
 |-- flightno: string (nullable = true)
 |-- flightdate: string (nullable = true)
 |-- chocksoff: timestamp (nullable = true)
 |-- chockson: timestamp (nullable = true)
 |-- domint: string (nullable = true)
 |-- manufacturerid: long (nullable = true)
 |-- manufacturer: string (nullable = true)
 |-- dataframeid: long (nullable = true)
 |-- dataframe: string (nullable = true)
 |-- aircrafttype: string (nullable = true)
 |-- aircraftmodel: string (nullable = true)
 |-- payload: string (nullable = true)
 |-- blocktime_hrs: string (nullable = true)
 |-- occurance: string (nullable = true)
 |-- typeid: 

In [ ]:
UNION_DF.count()

In [5]:
UNION_DF.createOrReplaceTempView('union_df')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
PilotInfoDF = spark.read.csv('s3://sjet-datamart-bucket/precog/pilot-info/', header=True)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
PilotInfoDF.createOrReplaceTempView('PilotInfoDF')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
final_df_df5 = spark.sql('''
                            SELECT 
                            CAST(ud.exceedanceid AS INT),
                            ud.exceedance,
                            CAST(ud.phaseid AS INT),
                            ud.phase,
                            ud.ecn,
                            ud.employeename,
                            ud.rank,
                            ud.tailno,
                            ud.departure,
                            ud.arrival,
                            ud.flightno AS FlightNumber,
                            ud.date as FLightDate ,
                            ud.chocksoff,
                            ud.chockson,
                            ud.domint,
                            CAST(ud.manufacturerid AS INT),
                            ud.manufacturer,
                            CAST(ud.dataframeid AS INT),
                            ud.dataframe,
                            ud.aircrafttype,
                            ud.aircraftmodel,
                            ud.payload,
                            ud.blocktime_hrs as `Block time`,
                            occurance as occurence,
                            CAST(typeid AS INT),
                            flag as type,
                            CAST(trigger_start_hours AS DECIMAL(8,2)) AS `Trigger Start Hours`,
                            CAST(trigger_end_hours AS DECIMAL(8,2)) AS `Trigger End Hours`,
                            CAST(occurence_start_hours AS DECIMAL(8,2)) AS `Occurence Start Hours`,
                            CAST(occurence_end_hours AS DECIMAL(8,2)) AS `Occurence End Hours`,
                            CAST(duration_min AS INT) AS  `Duration(Sec)`,
                            CAST(`BARO_COR_ALT` AS INT) AS `BARO COR ALT`,
                            CAST(`RADIO_HEIGHT` AS INT) AS `RADIO HEIGHT`,
                            CAST(`COMPUTED_AIRSPEED` AS FLOAT) AS `COMPUTED AIRSPEED`,
                            CAPT_DISPLAY_PITCH_ATTIUDE AS `CAPT. DISPLAY PITCH ATTIUDE`,
                            CAST(`CAPT_DISPLAY_HEADING` AS FLOAT) AS `CAPT. DISPLAY HEADING`,
                            CAST(`GROUNDSPEED` AS FLOAT) AS `GROUND SPEED`,
                            CAST(`VERTICAL_SPEED` AS INT) AS `VERTICAL SPEED`,
                            CAST(`FLAP_HANDLE_POSITION` AS INT) AS `FLAP HANDLE POSITION`,
                            CAST(`TORQUE_ENG_L` AS FLOAT) AS `LEFT ENGINE N1 \ TORQUE ENG L`,
                            CAST(`TORQUE_ENG_R` AS FLOAT) AS `RIGHT ENGINE N1 \ TORQUE ENG R`,
                            CAST('' AS INT) AS `VREF SPEED`,
                            MAIN_WOW AS `LEFT MAIN GEAR AIR/GND`,
                            '' AS `RIGHT MAIN GEAR AIR/GND`,
                            loadeddate AS LoadDate,
                            filename
                    FROM union_df ud
                    
                    
                ''')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
final_df_df5 \
  .write \
  .format("jdbc") \
  .mode("append") \
  .option("driver", "com.microsoft.sqlserver.jdbc.SQLServerDriver") \
  .option("url", "jdbc:sqlserver://10.150.72.22:1433;databaseName=precog") \
  .option("dbtable", "HighRateOfDescentSTG") \
  .option("user", "dmbigdata") \
  .option("password", "$p!Ce@bigData$db") \
  .save()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
UNION_DF.select('Flag').distinct().show(truncate = False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----+
|Flag |
+-----+
|Green|
|Amber|
|Red  |
+-----+

In [8]:
UNION_DF_1 = spark.sql('''

                        SELECT * FROM precogdb.df6_red_high_rate_of_descent
                        where date > '2021-10-31'
                        UNION
                        SELECT * FROM precogdb.df6_amber_high_rate_of_descent
                        where date > '2021-10-31'
                        UNION
                        SELECT * FROM precogdb.df6_green_high_rate_of_descent
                        where flightdate > '2021-10-31'
                        
                        ''')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
UNION_DF_1.createOrReplaceTempView('union_df_1')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
final_df_df6 = spark.sql('''
                            SELECT 
                            CAST(ud.exceedanceid AS INT),
                            ud.exceedance,
                            CAST(ud.phaseid AS INT),
                            ud.phase,
                            ud.ecn,
                            ud.employeename,
                            ud.rank,
                            ud.tailno,
                            ud.departure,
                            ud.arrival,
                            ud.flightno AS FlightNumber,
                            ud.date as FLightDate,
                            ud.chocksoff,
                            ud.chockson,
                            ud.domint,
                            CAST(ud.manufacturerid AS INT),
                            ud.manufacturer,
                            CAST(ud.dataframeid AS INT),
                            ud.dataframe,
                            ud.aircrafttype,
                            ud.aircraftmodel,
                            ud.payload,
                            ud.blocktime_hrs as `Block time`,
                            occurance as occurence,
                            CAST(typeid AS INT),
                            flag as type,
                            CAST(trigger_start_hours AS DECIMAL(8,2)) AS `Trigger Start Hours`,
                            CAST(trigger_end_hours AS DECIMAL(8,2)) AS `Trigger End Hours`,
                            CAST(occurence_start_hours AS DECIMAL(8,2)) AS `Occurence Start Hours`,
                            CAST(occurence_end_hours AS DECIMAL(8,2)) AS `Occurence End Hours`,
                            CAST(duration_min AS INT) AS  `Duration(Sec)`,
                            CAST(`BARO_COR_ALT` AS INT) AS `BARO COR ALT`,
                            CAST(`RADIO_HEIGHT` AS INT) AS `RADIO HEIGHT`,
                            CAST(`COMPUTED_AIRSPEED` AS FLOAT) AS `COMPUTED AIRSPEED`,
                            CAPT_DISPLAY_PITCH_ATTIUDE AS `CAPT. DISPLAY PITCH ATTIUDE`,
                            CAST(`CAPT_DISPLAY_HEADING` AS FLOAT) AS `CAPT. DISPLAY HEADING`,
                            CAST(`GROUNDSPEED` AS FLOAT) AS `GROUND SPEED`,
                            CAST(`VERTICAL_SPEED` AS INT) AS `VERTICAL SPEED`,
                            CAST(`FLAP_HANDLE_POSITION` AS INT) AS `FLAP HANDLE POSITION`,
                            CAST(`TORQUE_ENG_L` AS FLOAT) AS `LEFT ENGINE N1 \ TORQUE ENG L`,
                            CAST(`TORQUE_ENG_R` AS FLOAT) AS `RIGHT ENGINE N1 \ TORQUE ENG R`,
                            CAST('' AS INT) AS `VREF SPEED`,
                            WOW_MLG AS `LEFT MAIN GEAR AIR/GND`,
                            '' AS `RIGHT MAIN GEAR AIR/GND`,
                            loadeddate AS LoadDate,
                            filename
                    FROM union_df_1 ud
                    
                ''')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
final_df_df6 \
  .write \
  .format("jdbc") \
  .mode("append") \
  .option("driver", "com.microsoft.sqlserver.jdbc.SQLServerDriver") \
  .option("url", "jdbc:sqlserver://10.150.72.22:1433;databaseName=precog") \
  .option("dbtable", "HighRateOfDescentSTG") \
  .option("user", "dmbigdata") \
  .option("password", "$p!Ce@bigData$db") \
  .save()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
spark.stop()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…